# PlasticParcels Example
## The pathways and fate of Italian coastal plastic pollution
In this example, we will use `PlasticParcels` to run a basic simulation of microplastic pollution along the Italian coastline.

In [ ]:
# Library imports
from datetime import datetime, timedelta

# Parcels and PlasticParcels imports
import plasticparcels as pp

# Plotting imports
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
# Load the model settings
settings_file = 'docs/examples/lorenz_settings.json'
settings = pp.utils.load_settings(settings_file)

In [ ]:
# Create the simulation settings
settings['simulation'] = {
    'start_date': datetime.strptime('2019-01-10-00:00:00', '%Y-%m-%d-%H:%M:%S'), # Start date of simulation
    'runtime': timedelta(days=90),             # Runtime of simulation, use negative if releasing particles backwards in time
    'dt_write': timedelta(hours=12),             # Timestep of output
    'dt_timestep': timedelta(minutes=20),       # Timestep of advection
    }

# Overwrite some settings
settings['mode'] = '2D'
settings['use_biofouling'] = False
settings['use_stokes'] = True
settings['use_wind'] = True


In [ ]:
# Create the particle release settings
settings['release'] = {
    'initialisation_type': 'coastal',
    'country': 'Italy',
}

In [ ]:
# Create the plastic type settings
settings['plastictype'] = {
    'wind_coefficient' : 0.01, # Percentage of wind to apply to particles
    'plastic_diameter' : 0.001, # Plastic particle diameter (m)
    'plastic_density' : 1030., # Plastic particle density (kg/m^3)
}

In [ ]:
settings['ocean']['indices'] = {'lon':range(3300, 4000), 'lat':range(1850, 2400)}#, 'depth':range(0,2)}

In [ ]:
# Download the mask and release data
settings = pp.download_data.download_plasticparcels_dataset('NEMO0083', settings, 'input_data')

In [ ]:
# Create the fieldset
fieldset = pp.constructors.create_fieldset(settings)

In [ ]:
fieldset.computeTimeChunk()

ax=plt.subplot(111, projection=ccrs.PlateCarree())
ax.pcolormesh(fieldset.U.grid.lon, fieldset.U.grid.lat, fieldset.U.data[0,0,:,:], transform=ccrs.PlateCarree())
ax.coastlines()
plt.show()

In [ ]:
# Create the particleset
pset = pp.constructors.create_particleset_from_map(fieldset, settings)

In [ ]:
# Create the applicable kernels to the plastic particles
kernels = pp.constructors.create_kernel(fieldset)

In [ ]:
# Define the runtime, the timestepping, and the output frequency of the simulation from the settings
runtime = settings['simulation']['runtime']
dt_timestep = settings['simulation']['dt_timestep']
dt_write = settings['simulation']['dt_write']

In [ ]:
# Create the particle file where output will be stored
pfile = pp.ParticleFile('example_Italy_coast.zarr', pset, settings=settings, outputdt=dt_write)

In [ ]:
# Execute the simulation
pset.execute(kernels, runtime=runtime, dt=dt_timestep, output_file=pfile)

In [ ]:
import xarray as xr
ds = xr.open_zarr('example_Italy_coast.zarr')

In [ ]:
ds.plastic_amount

In [ ]:
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines(zorder=20)
ax.plot(ds['lon'].T, ds['lat'].T, transform=ccrs.PlateCarree())

plt.show()

In [ ]:
import numpy as np
ax = plt.subplot(111, projection=ccrs.PlateCarree())
ax.coastlines(zorder=20)
minval = np.min(ds['plastic_amount'].T[0,:])
maxval = np.max(ds['plastic_amount'].T[0,:])
for i in range(len(ds['plastic_amount'].T[0,:])):
    ax.plot(ds['lon'].T[:,i], ds['lat'].T[:,i], c=plt.cm.jet(ds['plastic_amount'].T[0,i]), transform=ccrs.PlateCarree())
plt.show()

In [ ]:
ds['plastic_amount'].T[0,:].values

In [ ]:
ds['plastic_amount'].T[0].values.shape

In [ ]:
kernels